In [2]:
import random
import tensorflow as tf
import os
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import random
import tensorflow as tf
import tensorflow as tf
import os
import numpy as np

def create_record(records_path, data_path):
    # 声明一个TFRecordWriter
    writer = tf.io.TFRecordWriter(records_path)
    # 读取图片信息，并且将读入的图片顺序打乱
    img_list = []
    filename = os.listdir(data_path)

    g = os.walk(data_path)
    training_set = []
    for path,dir_list,file_list in g:
        file_list.sort()
        for file_name in file_list:  
            image = tf.io.read_file(data_path + '/' + file_name)
            image = tf.image.decode_jpeg(image)
            training_set.append(image.numpy())   

    cnt = 0
    for i in range(len(training_set)):
        data_raw = training_set[i].tobytes()
        # 声明将要写入tfrecord的key值（即图片，标签）
        example = tf.train.Example(
           features=tf.train.Features(feature={
                "ID": tf.train.Feature(float_list=tf.train.FloatList(value=[float(file_list[i][:-4])])),
                "width": tf.train.Feature(float_list=tf.train.FloatList(value=[float(training_set[i].shape[0])])),
                "height": tf.train.Feature(float_list=tf.train.FloatList(value=[float(training_set[i].shape[1])])),
                'data_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data_raw]))
           }))
        # 将信息写入指定路径
        writer.write(example.SerializeToString())
        # 打印一些提示信息~
        cnt += 1

    writer.close()



In [ ]:
# 指定你想要生成tfrecord名称，图片文件夹路径，含有图片信息的txt文件

records_path = r'E:\NTNU2-1\imageRecognition\final project\trainingData.tfrecords'
data_path = r'E:\NTNU2-1\imageRecognition\final project\Train'

create_record(records_path, data_path)

In [ ]:
import tensorflow as tf
def _parse_function(example_proto):
    features = tf.io.parse_single_example(
        example_proto,
        features={
            "ID": tf.io.FixedLenFeature([], tf.float32),
            "width": tf.io.FixedLenFeature([], tf.float32),
            "height": tf.io.FixedLenFeature([], tf.float32),
            'data_raw': tf.io.FixedLenFeature([], tf.string)
        }
    )
    # 取出我们需要的数据（标签，图片）
    ID = features['ID']
    width = features['width']
    height = features['height']
    data = features['data_raw']
    data = tf.io.decode_raw(data, tf.uint8)
    data = tf.reshape(data, [width, height, 3])
    print(data)
    # 对标签以及图片作预处理
#     P1 = tf.reshape(P1, [1])
#     data = tf.reshape(data, [1, 128, 128, 128, 1])
    
    #img = tf.cast(img, tf.float32) * (1. / 255) - 0.5
    #label = tf.cast(label, tf.int32)
    return data
# 得到获取data batch的迭代器
def data_iterator(tfrecords):
    # 声明TFRecordDataset
    dataset =  tf.data.TFRecordDataset(tfrecords)
    dataset = dataset.map(_parse_function)
#     # 打乱顺序，无限重复训练数据，定义好batch size
#     dataset = dataset.shuffle(buffer_size=1000).repeat().batch(128)
    # 定义one_shot_iterator。官方上有许多类型的iterrator，这种是最简单的
    #iterator = dataset.make_one_shot_iterator()
    return dataset

# 指定TFrecords路径，得到training iterator。
train_tfrecords = '/home/csun001/finalProject/DevData.tfrecords'
dataset = data_iterator(train_tfrecords)

# # 使用方式举例
# with tf.Session(config= tfconfig) as sess:
#     tf.initialize_all_variables().run()
#     train_batch = train_iterator.get_next()
#     for step in xrange(50000):
#         train_x, train_y = sess.run(train_batch)

In [ ]:
import cv2
for i in enumerate(dataset):
    img_rgb = cv2.cvtColor(i[1].numpy(), cv2.COLOR_BGR2RGB)
    cv2.imshow('result',img_rgb )
    cv2.waitKey(0)
    break

In [28]:
def generateData(tfRecordDir):
    def _parse_function(example_proto):
        features = tf.io.parse_single_example(
            example_proto,
            features={
                "ID": tf.io.FixedLenFeature([], tf.float32),
                "width": tf.io.FixedLenFeature([], tf.float32),
                "height": tf.io.FixedLenFeature([], tf.float32),
                'data_raw': tf.io.FixedLenFeature([], tf.string)
            }
        )
        # 取出我们需要的数据（标签，图片）
        ID = features['ID']
        width = features['width']
        height = features['height']
        data = features['data_raw']
        data = tf.io.decode_raw(data, tf.uint8)
        data = tf.reshape(data, [width, height, 3])
        data = tf.cast(data, tf.float32)
        data = tf.image.resize(data, [256, 256], method='bicubic')
        data = (data - 127.5)/127.5


        return ID, data

    AUTOTUNE = tf.data.experimental.AUTOTUNE
    data = tf.data.TFRecordDataset(tfRecordDir)
    data = data.map(_parse_function, num_parallel_calls=AUTOTUNE)

    training_batch = data.batch(8, drop_remainder = True)
    training_batch = training_batch.prefetch(buffer_size = AUTOTUNE)
    return training_batch
    

In [29]:
tfRecordDir = '/home/csun001/finalProject/DevData.tfrecords'   
train_data = generateData(tfRecordDir)

In [30]:
r= np.zeros((1, 4*4*512))
for _, batch in enumerate(train_data):
#     result = en(batch)
    print(batch)
#     result = tf.reshape(result, [128, -1])
#     r = np.concatenate((r, result), axis=0)
    break

(<tf.Tensor: shape=(8,), dtype=float32, numpy=array([ 2.,  3.,  4., 22., 30., 35., 67., 78.], dtype=float32)>, <tf.Tensor: shape=(8, 256, 256, 3), dtype=float32, numpy=
array([[[[-0.5529412 , -0.5137255 , -0.48235294],
         [-0.5372549 , -0.49803922, -0.46666667],
         [-0.5294118 , -0.49019608, -0.45882353],
         ...,
         [-0.54995584, -0.31466177, -0.5028971 ],
         [-0.5294118 , -0.29411766, -0.48235294],
         [-0.56942207, -0.334128  , -0.52236325]],

        [[-0.5529412 , -0.5137255 , -0.48235294],
         [-0.5372549 , -0.49803922, -0.46666667],
         [-0.5294118 , -0.49019608, -0.45882353],
         ...,
         [-0.6309751 , -0.395681  , -0.5839163 ],
         [-0.52998155, -0.29468742, -0.48292273],
         [-0.5581325 , -0.3228384 , -0.5110737 ]],

        [[-0.5529412 , -0.5137255 , -0.48235294],
         [-0.53673613, -0.49752042, -0.46614787],
         [-0.5294118 , -0.49019608, -0.45882353],
         ...,
         [-0.5901713 , -0.35487717,

In [19]:
r

array([[   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [ 171.30381775,   14.76148987,   -9.32501221, ...,  287.605896  ,
         291.57562256,  234.58299255],
       [ 117.08260345,  -75.70037842,   68.2469635 , ...,  409.79293823,
         202.21463013,  190.74333191],
       ...,
       [ -22.75509644, -109.88082886,  -28.67158508, ...,  262.42956543,
         235.52700806,  330.47201538],
       [  20.0617981 , -169.49331665,   17.20485687, ...,  350.19055176,
         376.06567383,  238.3134613 ],
       [ -45.6131134 , -119.60964966,  -89.79498291, ...,  274.44973755,
         255.51002502,  253.23516846]])

In [8]:
from model import AE
import tensorflow.keras as keras
inputs = tf.keras.Input(shape=(256, 256, 3), name='img')

modelAE = AE(32) #32 is the hyperparameter of filter size
modelAE.load_weights("/home/csun001/finalProject/log/V2/trained_ckpt")
modelAE.call(inputs) 

m = keras.models.Model(inputs=inputs, outputs = modelAE.layers[-1].output)

en = keras.models.Model(inputs=inputs, outputs = m.layers[6].output)